# Automatic Evaluator

In [2]:
%%capture
#########################
# Requirements and Imports
#########################
!pip install ipywidgets
!pip install widgetsnbextension

from os import path
import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

In [3]:
reproducibility_field = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)
display(reproducibility_field)

Checkbox(value=True, description='Reproducibility')

In [4]:
reproducibility = reproducibility_field.value
if reproducibility:
    seed_field = widgets.IntText(
        value=0,
        placeholder=0,
        description="Seed:",
        disabled=False
    )
    display(seed_field)

IntText(value=0, description='Seed:')

In [5]:
# Set seeds for PRGs

reproducibility = reproducibility_field.value
if reproducibility:
    seed = seed_field.value
    print(seed)
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Use deterministic Algorithms
    torch.use_deterministic_algorithms(True)
    torch.utils.deterministic.fill_uninitialized_memory = True

0


## Game Selection

In [7]:
import ocatari
all_games = ocatari.core.AVAILABLE_GAMES
print(all_games)

['Adventure', 'AirRaid', 'Alien', 'Amidar', 'Assault', 'Asterix', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'BeamRider', 'Berzerk', 'Bowling', 'Boxing', 'Breakout', 'Carnival', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'DemonAttack', 'DonkeyKong', 'DoubleDunk', 'Enduro', 'FishingDerby', 'Freeway', 'Frogger', 'Frostbite', 'Galaxian', 'Gopher', 'Hero', 'IceHockey', 'Jamesbond', 'Kangaroo', 'KeystoneKapers', 'KingKong', 'Krull', 'KungFuMaster', 'MarioBros', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Pacman', 'Phoenix', 'Pitfall', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'RoadRunner', 'Seaquest', 'Skiing', 'SpaceInvaders', 'StarGunner', 'Tennis', 'TimePilot', 'UpNDown', 'Venture', 'VideoPinball', 'YarsRevenge', 'Zaxxon']


In [8]:
hack_games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('*.py') if x.is_file() and x.name != '__init__.py']

games_option = []
for g in all_games:
    if g.lower() in hack_games:
        games_option.append(g)


game_selector = widgets.Select(
    options = games_option,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)
game_selector.layout.width = '30%'
display(game_selector)

Select(description='Game:', index=21, layout=Layout(width='30%'), options=('Amidar', 'Assault', 'Asterix', 'At…

In [9]:
selected_game = game_selector.value
selected_game

'Pong'

# Model Selection

In [10]:
models = [m for m in Path('../models').glob('**/*.gz') if m.is_file() and selected_game.lower() in str(m).lower()]

model_selector = widgets.Select(
    options=models,
    value=None,
    description="Select Model:",
    disabled=False
)
model_selector.layout.width = '30%'
display(model_selector)


Select(description='Select Model:', layout=Layout(width='30%'), options=(PosixPath('../models/Pong/0/c51_class…

In [11]:
model_path = model_selector.value
model_path

PosixPath('../models/Pong/0/mdqn_modern_50M.gz')

# Add Run Arguments

In [12]:
# Human player option. If True, the game will wait for user input to play.
human_player = widgets.HBox([
    widgets.Text(value='--human', description='Let user play', disabled=True),
    widgets.Dropdown(options=[True, False], value=False, description='Type:', disabled=False)
])

display(human_player)

In [13]:
human_opt = '-hu' if human_player.children[1].value else None

In [14]:
import hackatari.core as core

avail_mod_for_game = core._available_modifications(selected_game).split('*')[1:]
modifies_list = [mod.split(':') for mod in avail_mod_for_game]

modifies_selector = widgets.VBox([
        widgets.HBox([
            widgets.Label(value=mod_name, layout={'width': '10%'}),
            widgets.Label(value=mod_info, layout={'width': '45%'}),
            widgets.Dropdown(options=[True, False], value=False, description='Value:',layout={'width': '15%'}, justify_content = 'flex-end')
        ], justify_content = 'space-between') for mod_name, mod_info in modifies_list
    ], layout={'width': '70%'})

display(modifies_selector)

In [15]:
mod_args = ' '.join([f'{mod.children[0].value.strip()}' for mod in modifies_selector.children if mod.children[2].value])
mod_args

'lazy_enemy up_drift'

# Run Game

In [16]:
## prepare the command
eval_script = Path(r"scripts/eval.py")
run_script = Path(r"scripts/run.py")

project_root = path.dirname(path.abspath(""))
print(f"Project Root: {project_root}")

# use posix_paths compatibile with the OS
eval_path = (Path(project_root) / eval_script).as_posix()
run_path = (Path(project_root) / run_script).as_posix()
model_path = Path(model_path).as_posix() if model_path is not None else ''

if human_opt:
    command = f"python {run_path} -g {selected_game} {human_opt}"
else:
    assert path.isfile(model_path), "For eval, agent model file is required"
    command = f"python {eval_path} -g {selected_game} -a {model_path} {mod_args}"

print("command: " + command)


Project Root: /mnt/c/Users/peng_/workspace/HackAtari
command: python /mnt/c/Users/peng_/workspace/HackAtari/scripts/eval.py -g Pong -a ../models/Pong/0/mdqn_modern_50M.gz lazy_enemy up_drift


In [17]:
# execute the command
!{command}

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
Loaded agent from ../models/Pong/0/mdqn_modern_50M.gz
Episode 1: Reward = 4.0
Episode 2: Reward = 3.0
Episode 3: Reward = 10.0
Episode 4: Reward = 11.0
Episode 5: Reward = 7.0
Episode 6: Reward = -3.0
Episode 7: Reward = 5.0
Episode 8: Reward = 14.0
Episode 9: Reward = -7.0
Episode 10: Reward = 10.0

Summary:
Agent: ../models/Pong/0/mdqn_modern_50M.gz
Total Episodes: 10
Average Reward: 5.40
Standard Deviation: 6.18
Min Reward: -7.0
Max Reward: 14.0
--------------------------------------
Traceback (most recent call last):
  File "/mnt/c/Users/peng_/workspace/HackAtari/scripts/eval.py", line 143, in <module>
    main()
  File "/mnt/c/Users/peng_/workspace/HackAtari/scripts/eval.py", line 98, in main
    agent, policy = load_agent(agent_path, env, "cpu")
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leqiaop1/peng_workspace/HackAtari/venv/lib/python3.11/site-packages/ocatari/utils.py", li

# Evaluation

In [38]:
# add util module to the path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

In [ ]:

from automatic_evaluator.utils import eval_withRun

eval_withRun(selected_game, [model_path], mod_args)




Loaded agent from ../models/Pong/0/mdqn_modern_50M.gz
--------------------------------------


In [40]:
%reload_ext aim


In [41]:

%aim up